# Importation Bibliothèque

In [1]:
import pandas as pd
import numpy as np
import json

from time import time
from collections import Counter

%pip install spotipy --upgrade
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

Note: you may need to restart the kernel to use updated packages.


# Spotify

## Compte

In [2]:
client_id     = "b608b0a22e5d4e5c882fc846ce59f9b7"
client_secret = "c99b4de8f5b74517b406544b96719fa5"


client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

## Fonction

In [3]:
def add_titre(col):
    return sp.track(col)['name']
def add_artist(col):
    return sp.track(col)['album']['artists'][0]['name']
def add_album(col):
    return sp.track(col)['album']['name']

def recup_liste_spotify(liste_m):
    '''
    Recupere les informations sur la musique à partir de son identifiant spotify
    Même si il y a un seul identifiant, le rentrer sous forme de liste
    '''
    df=pd.DataFrame({'track_id':liste_m})
    df['titre']=df['track_id'].apply(add_titre)
    df['album']=df['track_id'].apply(add_album)
    df['artist']=df['track_id'].apply(add_artist)
    return df

# Chargement des dictionnaires

Pour les dictionnaires utilisant les playlist pour la recommendation, ne charger qu'un seul des deux dictionnaires, il utilise les mêmes fonctions de recommendation.          

## Dictionnaire pour les genres musicaux

In [4]:
# Chargements des dictionnaire pour la recommendation avec les genres musicaux
times=time()

tf = open("Data/Recommendation_genre.json", "r")
dictionnaire= json.load(tf)
tf.close()

genre2music= dictionnaire['genre2music']
music2genre=dictionnaire['music2genre']
music2popularity=dictionnaire['music2popularity']
liste_music_genre=dictionnaire['liste_music']


del dictionnaire

times=time()-times
print("Temps d'éxécution :",round(times,2),"seconde")

Temps d'éxécution : 3.05 seconde


## Dictionnaire pour les playlists sur les musiques

In [4]:
# Chargements des dictionnaire pour la recommendation avec les playlists sur les musiques
times=time()

tf = open("Data/Recommendation_playlist_musique.json", "r")
dictionnaire= json.load(tf)
tf.close()

playlist2music= dictionnaire['playlist2music']

music2playlist= dictionnaire['music2playlist']
 
liste_music= dictionnaire['liste_music']
del dictionnaire

times=time()-times
print("Temps d'éxécution :",round(times,2),"seconde")

Temps d'éxécution : 3.76 seconde


## Dictionnaire pour le dataset playlist

In [4]:
# Chargements des dictionnaire pour la recommendation avec le dataset playlist
times=time()

tf = open("Data/Recommendation_playlist.json", "r")
dictionnaire= json.load(tf)
tf.close()

playlist2music= dictionnaire['playlist2music']

music2playlist= dictionnaire['music2playlist']
 
liste_music= dictionnaire['liste_music']
del dictionnaire

times=time()-times
print("Temps d'éxécution :",round(times,2),"seconde")

Temps d'éxécution : 35.22 seconde


# Fonction recommendation

In [5]:
def recherche(id_track,liste_m):
    '''
    Fonction cherchant à rechercher si l'identifiant spotify de la musique est bien présent 
    dans le jeu de donnée utilisé  
    '''
    if id_track in liste_m:
        return True
    else:
        return False
    

Ne charger que la ou les fonctions de recommendation qui correpondent au dictionnaire chargé

## Recommendation genre

In [8]:
def recommandation_musicale_genre(music,
                            liste_music=liste_music_genre,
                            genre2music=genre2music,
                            music2genre=music2genre,
                            music2popularity=music2popularity):
    '''
    Fonction permettant la recomendation d'une liste de musique en fonction d'une music en entrée
    
    music : musique à partir de laquel effectuer la recommendation
    liste_music : liste de musique dans laquel s'éffectue la recommendation
    genre2music : bibliothèque donnant pour chaque genre musicale les musiques correspondante
    music2genre : bibliothèque donnant pour chaque musique les genres musicaux correspondant
    music2popularity : bibliothèque donnant pour chaque musique la note de popularité correspondant
    
    '''
    
    if music not in liste_music:
        print("Identifiant de la musique incorrect")
        return 
    
    genres=music2genre[music]
    music_recommande=[]
    for genre in genres:
        music_recommande+=genre2music[genre]
    popularity=[]
    for track in list(Counter(music_recommande).keys()):
        popularity.append(music2popularity[track])
    music_recommande=pd.DataFrame({'track':list(Counter(music_recommande).keys()),
                                   'nombre':list(Counter(music_recommande).values()),
                                   'popularity':popularity}).sort_values(by=['nombre','popularity'],ascending=False)
    music_recommande=music_recommande[music_recommande['track']!=music]
    return list(music_recommande['track'].iloc[:10])

## Recommendation playlist

In [6]:
def recommandation_musicale(music,
                            liste_music=liste_music,
                            playlist2music=playlist2music,
                            music2playlist=music2playlist):
    '''
    Fonction permettant la recomendation d'une liste de musique en fonction d'une music en entrée
    
    music : musique à partir de laquel effectuer la recommendation
    liste_music : liste de musique dans laquel s'éffectue la recommendation
    playlist2music : bibliothèque donnant pour chaque playlist les musiques correspondante
    music2playlist : bibliothèque donnant pour chaque musique la playlist correspondante

    
    '''
    
    if music not in liste_music:
        print("Identifiant de la musique incorrect")
        return 
    
    playlists=music2playlist[music]
    music_recommande=[]
    for playlist in playlists:
        music_recommande+=playlist2music[str(playlist)]
    music_recommande=pd.DataFrame({'track':list(Counter(music_recommande).keys()),
                                   'nombre':list(Counter(music_recommande).values())}
                                 ).sort_values(by=['nombre'],ascending=False)
    music_recommande=music_recommande[music_recommande['track']!=music]
    return list(music_recommande['track'].iloc[:10])

In [7]:
def recommandation_playlist(liste_ecoute,
                            liste_music=liste_music,
                            playlist2music=playlist2music,
                            music2playlist=music2playlist):
    '''
    Fonction permettant la recomendation d'une liste de musique en fonction d'une music en entrée
    
    liste_ecoute : liste de musique à partir de laquel effectuer la recommendation
    liste_music : liste de musique dans laquel s'éffectue la recommendation
    playlist2music : bibliothèque donnant pour chaque playlist les musiques correspondante
    music2playlist : bibliothèque donnant pour chaque musique la playlist correspondante

    
    '''
    
    
    for music in liste_ecoute:
        if music not in liste_music:
            print("Identifiant d'une des musique incorrect")
            print("Identifiant :",music)
            return 
    music_recommande=[]
    for music in liste_ecoute:
        playlists=music2playlist[music]
    
        for playlist in playlists:
            music_recommande+=playlist2music[str(playlist)]
    music_recommande=pd.DataFrame({'track':list(Counter(music_recommande).keys()),
                                   'nombre':list(Counter(music_recommande).values())}
                                 ).sort_values(by=['nombre'],ascending=False)
    music_recommande=music_recommande[~music_recommande['track'].isin(liste_ecoute)]
    return list(music_recommande['track'].iloc[:10])

# Utilisation des fonctions


In [8]:
recherche('6nTiIhLmQ3FWhvrGafw2zj',liste_music)

True

In [9]:
music=['57bgtoPSgt236HzfBOd8kj','3jUTjCISntIUFL8jnAjzgc','1DeaByFASTvBxtYANOyFXy','7LRMbd3LEoV5wZJvXT1Lwb','2QSAj76Ba6aMFX9RlXdUdO','6nTiIhLmQ3FWhvrGafw2zj']
recup_liste_spotify(music)


,track_id,titre,album,artist
0,57bgtoPSgt236HzfBOd8kj,Thunderstruck,The Razors Edge,AC/DC
1,3jUTjCISntIUFL8jnAjzgc,Fallen Leaves,Billy Talent II,Billy Talent
2,1DeaByFASTvBxtYANOyFXy,Surrender,Billy Talent II,Billy Talent
3,7LRMbd3LEoV5wZJvXT1Lwb,T.N.T.,High Voltage,AC/DC
4,2QSAj76Ba6aMFX9RlXdUdO,J'ai demandé à la lune,Paradize,Indochine
5,6nTiIhLmQ3FWhvrGafw2zj,American Idiot,American Idiot,Green Day


In [10]:
recup_liste_spotify(recommandation_playlist(music))

,track_id,titre,album,artist
0,08mG3Y1vljYA6bvDt4Wqkj,Back In Black,Back In Black,AC/DC
1,2zYzyRzz6pRmhPzyfMEC8s,Highway to Hell,Highway to Hell,AC/DC
2,7o2CTH4ctstm8TNelqjb51,Sweet Child O' Mine,Appetite For Destruction,Guns N' Roses
3,0bVtevEgtDIeRjCJbK3Lmv,Welcome To The Jungle,Appetite For Destruction,Guns N' Roses
4,2SiXAy7TuUkycRVbbWDEpo,You Shook Me All Night Long,Back In Black,AC/DC
5,1AhDOtG9vPSOmsWgNW0BEY,Bohemian Rhapsody - Remastered 2011,A Night At The Opera (2011 Remaster),Queen
6,3YBZIN3rekqsKxbJc9FZko,Paradise City,Appetite For Destruction,Guns N' Roses
7,4CJVkjo5WpmUAKp3R44LNb,Sweet Home Alabama,Second Helping,Lynyrd Skynyrd
8,5MxNLUsfh7uzROypsoO5qe,Dream On,Aerosmith,Aerosmith
9,4bHsxqR3GMrXTxEPLuK5ue,Don't Stop Believin',Escape,Journey


In [11]:
music='1MQTmpYOZ6fcMQc56Hdo7T'
recup_liste_spotify([music])

,track_id,titre,album,artist
0,1MQTmpYOZ6fcMQc56Hdo7T,Sleep Now In the Fire,The Battle Of Los Angeles,Rage Against The Machine


In [12]:
recup_liste_spotify(recommandation_musicale(music))

,track_id,titre,album,artist
0,59WN2psjkt1tyaxjspN8fp,Killing In The Name,Rage Against The Machine - XX (20th Anniversar...,Rage Against The Machine
1,0tZ3mElWcr74OOhKEiNz1x,Bulls On Parade,Evil Empire,Rage Against The Machine
2,7lmeHLHBe4nmXzuXc0HDjk,Testify,The Battle Of Los Angeles,Rage Against The Machine
3,1wsRitfRRtWyEapl0q22o8,Guerrilla Radio,The Battle Of Los Angeles,Rage Against The Machine
4,5YBVDvTSSSiqv7KZDeUlXA,Renegades Of Funk,Renegades,Rage Against The Machine
5,6ZU9RJIZ0fNaFuQM57bDIA,Bombtrack,Rage Against The Machine,Rage Against The Machine
6,5ghIJDpPoe3CfHMGu71E6T,Smells Like Teen Spirit,Nevermind (Remastered),Nirvana
7,5V3mdRI2yQxbSsJGDPc5lD,Chop Suey!,Toxicity,System Of A Down
8,1bdXMstfxFWYSkEFTnJMoN,Enter Sandman,Metallica,Metallica
9,5UWwZ5lm5PKu6eKsHAGxOk,Everlong,The Colour And The Shape,Foo Fighters
